### INTRODUCTION: 

##### The visualization for this notebook uses Lonboard, which is a new Python library for fast, interactive geospatial vector data visualization for big raster or vector files in Jupyter. By utilizing new technologies like GeoArrow and GeoParquet in conjunction with GPU-based map rendering, lonboard aims to enable visualizing large geospatial datasets interactively through a simple interface. 

CODE PREPARED BY:

-- Rajat Shinde, JPL NASA
-- Alex Mandel, Development Seed

CODE ADAPTED BY:

-- Paromita Basak, University of Maryland

#### TO MOVE TO PUBLIC INTERFACE, DO THE FOLLOWING: 

1. Creating executed notebook

python -m nbconvert /projects/ADE_biomass_harmonization/NASA_CMS/NASA_CMS_2023/Cambodia/GEDI_InteractiveNotebook.ipynb --to ipynb --stdout --execute > /projects/ADE_biomass_harmonization/NASA_CMS/NASA_CMS_2023/Cambodia/GEDI_InteractiveNotebook_executed.ipynb

2. Create shared notebook

pip install nbss-upload

nbss-upload /projects/ADE_biomass_harmonization/NASA_CMS/NASA_CMS_2023/Cambodia/GEDI_InteractiveNotebook_executed.ipynb

In [1]:
!pip install lonboard pyogrio
!pip install localtileserver ipyleaflet

In [2]:
import geopandas as gpd
from lonboard import viz
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from lonboard import Map, BitmapLayer, BitmapTileLayer, PointCloudLayer, PolygonLayer, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap
import json
from palettable.colorbrewer.sequential import YlGnBu_7, Greens_3
from palettable.colorbrewer.diverging import Spectral_9
from matplotlib.colors import Normalize
import httpx

**Adding NFI Data**

In this first section we visualize Cambodia's National Forest Inventory(NFI) data over Mondulkiri Srepok National Park.

In [3]:
##Define NFI file path
path = "/projects/shared-buckets/nehajo88/Data/NASA_CMS_2023/CAMBODIA/KHM_AGBD.gpkg"

In [4]:
#Read file using geopandas and pyogrio
NFI_gdf = gpd.read_file(path, engine="pyogrio")

In [ ]:
#Create a scatter plot layer from a GeoPandas DataFrame (NFI_gdf) and adds it to a map
NFI_layer = ScatterplotLayer.from_geopandas(NFI_gdf)
m = Map(layers=[NFI_layer])

In [ ]:
##Sets the fill color of the GEDI_original_layer scatter plot to a specific RGB color value [200, 100, 100].
NFI_layer.get_fill_color = [200, 100, 100]

In [ ]:
#Prints the file head for seeing column names which will be needed in next step
NFI_gdf.head()

Now, we would like to plot data based on a statistic. Let's plot based on the value of the 'AGBD.PLOT' column. We will use a linear colormap excluding the lowest and highest 10 percentiles of the data.

First, we'll find what the values of the 10th and 90th percentile are:

In [ ]:
#Calculate the 10th percentile (q10) and the 90th percentile (q90) of the 'agbd' column in the gedi_gdf GeoPandas DataFrame.
q10 = NFI_gdf['AGBD.PLOT'].quantile(.1)
q90 = NFI_gdf['AGBD.PLOT'].quantile(.9)

Then we can construct a "normalizer" based on Normalize that will do linear rescaling between the two values we supply:

In [ ]:
normalizer = mpl.colors.Normalize(q10, q90)

Calling this normalizer on our data will return a scaled version ranging between 0 and 1:

In [ ]:
normalized_agbd = normalizer(NFI_gdf['AGBD.PLOT'])
normalized_agbd

Here we construct color values by taking a Matplotlib colormap and calling it on our normalized values. Ensure you use bytes=True when calling the colormap object so that the output colors range from 0-255 and not 0-1.

In [ ]:
# Ensure lengths match
if len(normalized_agbd) == len(NFI_gdf):
    # Apply the viridis colormap
    viridis_colormap = plt.get_cmap("viridis")
    colors = viridis_colormap(normalized_agbd, bytes=True)
    NFI_layer.get_fill_color = colors
    NFI_layer.radius_min_pixels = 5
else:
    raise ValueError("normalized_agbd length does not match the length of Improved_gedi_gdf")

The map should now have updated to show points colored by their agbd value. To remind ourselves what the high and low values are, we can inspect the colormap object itself:

In [ ]:
viridis_colormap

For showing the plots against a basemap, here we call upon a basemap from ESRI which can be used on lonboard visualization of the plots.

In [ ]:
#Voyager = 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'

In [ ]:
ESRI_basemap = BitmapTileLayer(
    data="https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    tile_size=256,
    max_requests=-1,
    min_zoom=0,
    max_zoom=19,
)
m = Map(ESRI_basemap)

In [ ]:
m = Map([ESRI_basemap,NFI_layer])
m

**GEDI Original L4A Shots Visualization**

The NASA GEDI L4A data offers predictions of aboveground biomass density (AGBD in Mg/ha) and provides uncertainty estimates for each sampled geolocated laser footprint. The footprints are located within the global latitude band observed by the International Space Station (ISS), nominally between 51.6 degrees N and S, and cover the period from April 18, 2019, to March 16, 2023.

This section's code reads and visualizes all the GEDI L4A shots in entire country of Cambodia using libraries such as geopandas, pyogrio and lonboard. The codes follow similar steps as described in the first section.

In [ ]:
path = "/projects/shared-buckets/leitoldv/gedi_subsetting_CAMBODIA/DATA/KHM_L4A.gpkg"
gedi_gdf = gpd.read_file(path, engine="pyogrio")
GEDI_original_layer = ScatterplotLayer.from_geopandas(gedi_gdf)
m = Map(layers=[GEDI_original_layer])

In [ ]:
GEDI_original_layer.get_fill_color = [200, 100, 100]

In [ ]:
gedi_gdf.head()

In [ ]:
m

In [ ]:
q10 = gedi_gdf['agbd'].quantile(.1)
q90 = gedi_gdf['agbd'].quantile(.9)

In [ ]:
normalizer = mpl.colors.Normalize(q10, q90)

In [ ]:
normalized_agbd = normalizer(gedi_gdf['agbd'])
normalized_agbd

In [ ]:
viridis_colormap = mpl.colormaps["viridis"]
GEDI_original_layer.get_fill_color = viridis_colormap(normalized_agbd, bytes=True)
GEDI_original_layer.radius_min_pixels = 1

In [ ]:
viridis_colormap

In [ ]:
m = Map(layers=[ESRI_basemap, GEDI_original_layer])
m

**Adding Improved GEDI L4A**

In this part we visualize the improved GEDI data over Mondulkiri Srepok National Park which was derived from our team's implementation of improved model for the area. We used additional field data (previously not available) to train our biomass model in Asian Evergreen Broadleaf Forests (EBT_As) which resulted in improved AGBD estimates for this region. 

The codes follow similar steps as described in the first section.

In [ ]:
path = "/projects/shared-buckets/pbasak/Cambodia_Notebook/NPAs_L4A_improved.gpkg"

In [ ]:
Improved_GEDI_gdf = gpd.read_file(path, engine="pyogrio")

In [ ]:
Improved_GEDI_gdf.head()

In [ ]:
Improved_GEDI_layer = ScatterplotLayer.from_geopandas(Improved_GEDI_gdf)
m = Map(layers=[Improved_GEDI_layer])

In [ ]:
m

In [ ]:
q10 = Improved_GEDI_gdf['AGBD_1'].quantile(.1)
q90 = Improved_GEDI_gdf['AGBD_1'].quantile(.9)

In [ ]:
normalizer = mpl.colors.Normalize(q10, q90)

In [ ]:
normalized_agbd = normalizer(Improved_GEDI_gdf['AGBD_1'])
normalized_agbd

In [ ]:
# Ensure lengths match
if len(normalized_agbd) == len(Improved_GEDI_gdf):
    # Apply the viridis colormap
    viridis_colormap = plt.get_cmap("viridis")
    colors = viridis_colormap(normalized_agbd, bytes=True)
    Improved_GEDI_layer.get_fill_color = colors
    Improved_GEDI_layer.radius_min_pixels = 1
else:
    raise ValueError("normalized_agbd length does not match the length of Improved_gedi_gdf")

In [ ]:
viridis_colormap

In [ ]:
m = Map(layers=[ESRI_basemap, Improved_GEDI_layer,NFI_layer])
m

**CCI Biomass Layer**

The ESA's Climate Change Initiative (CCI) Biomass project aimed to produce global maps of above-ground biomass for 2010, 2017, and 2018 to quantify biomass changes. The mapping is done with a 100-meter grid spacing, aiming for a relative error of less than 20 percent in areas where the above-ground biomass (AGB) exceeds 50 Mg ha-1. 

For visualizing the CCI Biomass layer, we will use BitmapTileLayer which renders tiles dynamically generated by TiTiler.

In [ ]:
titiler_endpoint = "https://titiler.maap-project.org"

In [ ]:
CCI_product_url = "s3://maap-ops-workspace/shared/pbasak/Cambodia_Notebook/CCI_AOImasked.tif"

In [ ]:
CCI_product_tile_url = "https://titiler.maap-project.org/cog/tiles/{z}/{x}/{y}?url=s3://maap-ops-workspace/shared/pbasak/Cambodia_Notebook/CCI_AOImasked.tif&rescale=0,60&colormap_name=viridis"

In [ ]:
r_CCI = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": CCI_product_url,
    }
).json()

bounds_CCI = r_CCI["bounds"]
print(bounds_CCI)

In [ ]:
CCI_product_layer = BitmapTileLayer(
    data=CCI_product_tile_url,
    tile_size=148,
    max_requests=-1,
    extent=bounds_CCI
)

In [ ]:
viridis_colormap

In [ ]:
m1 = Map([layer, CCI_product_layer, Improved_GEDI_layer, NFI_layer])
m1